<img src="images/zapata.png" width="100px">

# CUSP Code Demonstration
    
Compressed Unsupervised State Preparation (CUSP) is a method for building more efficient quantum circuits by using a quantum autoencoder.  The protocol performs a kind of circuit synthesis that, if training is successful, results in a more compact circuit.  Since typically shorter-depth circuits are less prone to noise on a real quantum computer, this tool gives the opportunity to make a more accurate state preparation, resulting in better accuracy for a quantum computation.  In this demo, we will use the example of improving a circuit which computes the ground state energies of molecular hydrogen at various bond lengths.

CUSP has three stages of learning (with an optional fourth stage that is not included in this demo).  The first stage is to simply train (e.g. using a circuit ansatz) the circuit that we would like to improve. In the second stage, we will train a quantum autoencoder to compress the output of Stage 1 into a representation on a smaller number of qubits (called the "latent space").  In the last stage, we choose a small variational circuit in the latent space, apply the decoder that we learned from Stage 2, and try to optimize the parameters to generate the best state we can.  If that sounds a bit abstract, don't worry!  We will step you more specifically through each of the stages below.

Please note that this is the short version of the demo, which is a bit more elegant to look at and use, but does not get into the details of the code. A more detailed tutorial is coming soon!

Let's start by importing some necessary modules.  In this short demo, almost all of the code is hidden in these files so you don't have to see it.  Note that, in this tutorial, we are implementing all of the code using Google's quantum simulator, Cirq.

In [1]:
# Import modules
import numpy as np
from scipy.optimize import minimize

## Background

For this demo, we will look to improving a circuit ansatz which is used for the variational quantum eigensolver (VQE) algorithm.  VQE is a variational algorithm that computes the approximate ground state energy of a molecule at a particular configuration.  Here, we will consider a very simple example of molecular hydrogen (H2) with a variable bond length (the bond length is the distance between the two hydrogen atoms).  Since the ground state energy of H2 changes as a function of the bond length, we will have to choose which bond lengths we would like to train on and compute.


<img src="images/H2_curve.png" width="500px">


<center>Bond dissociation curve for molecular hydrogen in the minimal basis (STO-3G)</center>


For simplicity in this demo, the variational circuits have been chosen for you in each stage, which we will detail more later.  You will be left to choose which parameters of those circuits to fix, and which to optimize over.

## Settings for CUSP

This is the general environment in which CUSP will run, including strength of noise, number of statistical samples, and the set of training examples to consider.  In order to give a fair analysis of the algorithm, these settings will apply to all stages of the circuit.

`num_trials` determines how many times we will run a circuit to gather our statistics. Optimization routines may struggle to converge if `num_trials` is small and noise is large.  However, the time to run the simulation is largely determined by the value of `num_trials` so we recommend `num_trials = 1000` as an upper bound if the noise is significant.  If the noise is subtle, `num_trials = 600` will likely be sufficient.

Setting `include_gate_noise` to `True` allows you to include noise during the simulation.  Note that turning this on will increase the simulation time roughly by the scale of `num_trials`, because we will run that many instances of the circuit to get our measurement statistics for just one run.  To see a substantial difference between the original circuit and the CUSP optimized circuit, you will need to set `gate_noise = True`.  However, to simply see how the CUSP protocol operates, we recommend you set `gate_noise = False` since this will dramatically increase the speed of the simulation.

`noise_level` determines the amount of (dephasing) noise to include during the simulations.  The value of `noise_level` is associated with the probability of having an error after each operation, so this parameter ranges between 0 (no noise) and 1 (an error after every gate). By default, Cirq's noisy simulations run with a noise level of `0.001`.  Because the original circuit we are trying to optimize is already a bit short, we recommend `noise_level = 0.002` to see a marked difference between the original circuit and the CUSP-improved circuit.  Note that very high levels of noise may prevent the protocol from working since the autoencoder would rarely get reliable input to train on.

`bond_lengths` determines what our training set will be throughout the CUSP protocol.  For the autoencoder to learn a good representation of the ground states, the training set should include a range of bond length examples.  Note that a larger training set will correspond to a longer simulation. We recommend `bond_lengths = [1.0, 1.5, 2.0, 2.5]` as a reasonable set.  The set of valid bond lengths to choose from are in the range of `0.2` to `4.1` in increments of `0.1`.

In [2]:
# Settings
num_trials = 600
include_gate_noise = False
noise_level = 0.002

bond_lengths = [1.0, 1.5, 2.0, 2.5]

user_settings = np.array([True, include_gate_noise, noise_level,
                          num_trials, bond_lengths], dtype=object)
np.save('data/user_settings', user_settings)

We now import some necessary files with the settings you have chosen.

In [3]:
# User settings for CUSP
import settings

# Modules containing subroutines to set up cost functions at each stage
import cusp_stage1
import cusp_stage2
import cusp_stage3

# Modules containing subroutines to set up optimization at each stage
import stage1_opt
import stage2_opt
import stage3_opt

Note: If you change the settings above, you will need to restart your Jupyter notebook kernel and run the above cells again.

## CUSP Stage 1: Preparation of Training Set

<img src="images/stage1_alg.png" width="500px">

Recall that in Stage 1 of this example, we will be trying to improve a circuit that prepares ground states of H2 for VQE.  The chosen circuit was generated from a circuit ansatz called "unitary coupled-cluster", and has just a single variational parameter, `alpha`, which is the degree of rotation of a Z gate (circled below).  The circuit looks like this:

<img src="images/stage1.png" width="500px">


The code below will execute an optimization by trying to find the value of `alpha` which minimizes the ground state energy, and give a comparison to the actual ground state energy.

In [4]:
%%time

print('#### STAGE 1 OF CUSP NOW RUNNING ####\n')

# Lists to store energies
check_energies = []       # Energies of FCI/exact wavefunctions
stage1_energies = []      # Energies of VQE wavefunctions

# Run thru bond lengths (or the training set)
for bond_length in bond_lengths:
    
    # Run VQE calculation for each training point/state
    opt_stage1_params = stage1_opt.run_state_preparation_optimization(bond_length)
    print('Optimizing for bond length {0} ... '
          'Optimal parameter setting is: {1}'.format(bond_length, opt_stage1_params))

    # Compute and store energies to check results
    exact_energy = settings.fetch_ground_energy(bond_length)
    check_energies.append(exact_energy)
    opt_energy = cusp_stage1.compute_stage1_cost_function(opt_stage1_params,
                                                          bond_length,
                                                          n_repetitions=1,
                                                          exact=True,
                                                          noisy=include_gate_noise)
    stage1_energies.append(opt_energy)
    
    # Display stage 1 results
    print('Exact ground state energy               : {}'.format(exact_energy))
    print('VQE optimized energy                    : {}'.format(opt_energy))
    print('Energy difference (absolute value)      : {}\n'.format(
            np.abs(opt_energy - exact_energy)))
    
    # Save these optimized VQE parameters into numpy arrays
    np.save('data/stage1_param_{}'.format(bond_length), opt_stage1_params)

#### STAGE 1 OF CUSP NOW RUNNING ####

Optimizing for bond length 1.0 ... Optimal parameter setting is: [2.11243634]
Exact ground state energy               : -1.1011503293035882
VQE optimized energy                    : -1.101150146492625
Energy difference (absolute value)      : 1.8281096325623025e-07

Optimizing for bond length 1.5 ... Optimal parameter setting is: [0.23152631]
Exact ground state energy               : -0.9981493524136991
VQE optimized energy                    : -0.9981493519938817
Energy difference (absolute value)      : 4.19817403063405e-10

Optimizing for bond length 2.0 ... Optimal parameter setting is: [0.36039421]
Exact ground state energy               : -0.9486411117424077
VQE optimized energy                    : -0.9486409770335681
Energy difference (absolute value)      : 1.347088396386198e-07

Optimizing for bond length 2.5 ... Optimal parameter setting is: [0.43977224]
Exact ground state energy               : -0.9360549198442759
VQE optimized energy 

You will likely observe that if `gate_noise = False`, these energies are very close to each other; if `gate_noise = True`, the energy difference typically gets larger as `noise_level` increases.

## CUSP Stage 2: Training the Quantum Autoencoder (QAE)

<img src="images/stage2_alg.png" width="500px">

Now that we have trained an initial circuit, we wish to apply a quantum autoencoder to the output of the previous state.  Before we get ahead of ourselves, let's start by loading the parameters of the previous circuit.

In [5]:
# Load the optimized VQE parameters from stage 1 of CUSP
stage1_param_list = []

for bond_length in bond_lengths:
    stage1_param = np.load('data/stage1_param_{}.npy'.format(bond_length))
    stage1_param_list.append(stage1_param)

The autoencoder's job in Stage 2 is to compress the quantum information from the initial state preparation down to (in this example) just a single qubit.  We know this is in principle possible because we had only one parameter in the original state preparation.  The challenge at this stage is to find an autoencoder circuit that is general enough to accomplish this task, but is not as deep as the original circuit.  For simplicity in this demo, we have chosen the circuit for you -- it is a set of three "parameterized CNOT" gates.  A regular CNOT, on Google's quantum hardware, is compiled in the following way:

<img src="images/cnot.png" width="400px">

We are going to replace these gates with the native parameterized gates that they are composed of on Google's hardware: we'll replace the Y gate with a variable W gate, the Z gate with a variable Z gate, and the CZ gate with a variable controlled-Z rotation.  The autoencoder circuit with the optimal parameters then looks like:

<img src="images/stage2.png" width="700px">

The entire Stage 2 circuit looks like:

<img src="images/circuit_pic.png" width="1100px">

Note that the bottom qubit will contain the latent space, while the top three are the reference qubits which we will use to train the autoencoder.  The training is performed by varying the parameters of the circuit and making repeated measurements of the reference qubits, attempting to maximize the frequency that $|000\rangle$ is measured.  If the autoencoder is able to find a parameter setting where $|000\rangle$ is achieved with high fidelity, it means that these qubits have been decoupled (or disentangled) from the remaining qubit.  As a result, all of the quantum information in the circuit must be possessed by the single qubit in the latent space, and so the autoencoder has succeeded in its task.

In the near future, we will update this demo with the ability to manually fix gates.  For now, the optimization will automatically search over all parameters.

The following code block will run the quantum autoencoder training.  Note a few options in the first few lines of the block, `threshold` and `n_qae_trials`.  Because the optimization routine can fail to give the global optimum if it gets caught in a local minimum, these settings allow the search to automatically restart up to `n_qae_trials` many times if the error of the autoencoder (computed as $1 - \text{Fidelity}$) in each case is above `threshold`.  If `threshold` is set too large, the autoencoder will be allowed to terminate after finding a local optimimum, and so will not perform well in Stage 3.  As `noise_level` increases, the rate of successfully training the autoencoder tends to decrease slightly.  Note that as `noise_level` increases, the minimum error of the training will also increase, but this minimum will still usually correspond to an optimal setting for the parameters.  Keep this in mind if you are running the simulation with a high `noise_level`, since you may wish to increase `threshold` to ensure that you are not asking the autoencoder for too much accuracy during training.  

In [6]:
%%time

print('Stage 2 using the following bond lengths for training: {}\n'.format(bond_lengths))

# QAE settings
threshold = 0.1
n_qae_trials = 25

print('#### STAGE 2 OF CUSP NOW RUNNING ####\n')
opt_qae_params = stage2_opt.run_qae_optimization(training_states=stage1_param_list,
                                                 n_repetitions=num_trials,
                                                 exact=True,
                                                 noisy=include_gate_noise)

# Repeat optimization of QAE circuit while error value is above threshold
iter_count = 0
while stage2_opt.compute_avg_fid_proxy(params=opt_qae_params,
                                       training_states=stage1_param_list,
                                       n_repetitions=num_trials,
                                       exact=True,
                                       noisy=include_gate_noise) > threshold:
    if iter_count >= n_qae_trials:
        print('Surpassed the QAE iteration limit. Exiting loop.')
        break
    
    print('Trial {}: Quantum autoencoder learning had low fidelity. '
          'Trying again.'.format(iter_count))
    
    opt_qae_params = stage2_opt.run_qae_optimization(training_states=stage1_param_list,
                                                     n_repetitions=num_trials,
                                                     exact=True,
                                                     noisy=include_gate_noise)
    iter_count += 1

# Compute error of optimized QAE circuit
err = stage2_opt.compute_avg_fid_proxy(opt_qae_params, training_states=stage1_param_list,
                                       n_repetitions=num_trials, exact=True,
                                       noisy=include_gate_noise)
print('Quantum autoencoder learning succeeded with error : {}'.format(err))

# Save QAE results
np.save('data/stage2_param', opt_qae_params)
print('')

Stage 2 using the following bond lengths for training: [1.0, 1.5, 2.0, 2.5]

#### STAGE 2 OF CUSP NOW RUNNING ####

Quantum autoencoder learning succeeded with error : 6.070367364330309e-08

CPU times: user 33.1 s, sys: 247 ms, total: 33.4 s
Wall time: 33.7 s


Once the training from Stage 2 is successful, the optimal parameters that are found are passed on to Stage 3.  These parameters are then used to construct the _decoder_ part of the quantum autoencoder, which is conveniently just the hermitian conjugate of the encoder.  In this example, for an optimal CNOT encoding, the decoder simply looks like:

<img src="images/decoder.png" width="150px">

## CUSP Stage 3: Generative Model Search

<img src="images/stage3_alg.png" width="500px">

We now address the final stage of CUSP wherein we construct the final circuit which will ideally produce a more accurate ground state energy than the original.  To do so, we need to search through the latent space to find a quantum state which approximates the compressed state at the end of Stage 2.  We will then use the autoencoder parameters found in the previous stage to construct a decoder which maps the state in the latent space back to the desired ground state of molecular hydrogen.  To optimize the latent space parameters, we will look to minimize the energy as we did in Stage 1.  In other words, we will perform VQE on our new circuit ansatz.

Because our latent space is only one qubit in width, we have a conveniently small space to search. We will again choose the circuit for you: a two-parameter W gate followed by a parameterized Z gate.  The entire latent space search (circled in the figure) and decoder circuit looks like:

<img src="images/stage3.png" width="250px">


As in Stage 2, in the near future we will update this demo with the ability to manually fix gates.  For now, the optimization will automatically search over all parameters.

The code block below now optimizes the latent space circuit, attempting to minimize the final state's energy.  The final result is compared to the true ground state energy, and to the energy that the original circuit found in Stage 1.  Because CUSP is designed to improve the gate depth, you will observe a significant improvement in the energy over the original circuit if `gate_noise=True`.  Otherwise, all of these energies are likely to be very close to one another.

In [7]:
%%time

# Optimal parameters from Stage 2
print('Parameters used from Stage 2: {}\n'.format(opt_qae_params))

print('#### STAGE 3 OF CUSP NOW RUNNING ####\n')

stage3_energies = []
cusp_params = {}

for i, bond_length in enumerate(bond_lengths):
    
    # Initialize parameters
    half_turn_min = 0
    half_turn_max = 2
    init_params = np.random.uniform(low=half_turn_min,
                                    high=half_turn_max,
                                    size=stage3_opt.num_param)

    # Optimization using Nelder-Mead
    stage3_fcn = lambda x: stage3_opt.stage3(x, bond_length, n_repetitions=num_trials)
    res = minimize(stage3_fcn, init_params, args=(),
                   method='Nelder-Mead', tol=None, 
                   options={'disp': False, 'maxiter': None, 'xatol': 0.001,
                            'return_all': False, 'fatol': 0.001})
    opt_cusp_param = res.x
    cusp_params[bond_length] = opt_cusp_param
    cusp_energy = cusp_stage3.run_sim_repetitions_stage3(*opt_cusp_param,
                                                         bond_length,
                                                         n_repetitions=1,
                                                         exact=True,
                                                         noisy=include_gate_noise)
    stage3_energies.append(cusp_energy)
    
    print('Bond length                             : {}'.format(bond_length))
    print('CUSP optimized energy                   : {}'.format(cusp_energy))
    print('Stage 1 energy                          : {}'.format(stage1_energies[i]))
    print('Exact energy                            : {}'.format(check_energies[i]))
    print('Energy difference (Stage 1 vs. exact)   : {}'.format(
            np.abs(stage1_energies[i] - check_energies[i])))
    print('Energy difference (CUSP    vs. exact)   : {}\n'.format(
            np.abs(cusp_energy - check_energies[i])))

Parameters used from Stage 2: [0.25003509 0.76861604 0.99994205 0.99991797]

#### STAGE 3 OF CUSP NOW RUNNING ####

Bond length                             : 1.0
CUSP optimized energy                   : -1.1011500162985328
Stage 1 energy                          : -1.101150146492625
Exact energy                            : -1.1011503293035882
Energy difference (Stage 1 vs. exact)   : 1.8281096325623025e-07
Energy difference (CUSP    vs. exact)   : 3.130050554300823e-07

Bond length                             : 1.5
CUSP optimized energy                   : -0.9981486196149264
Stage 1 energy                          : -0.9981493519938817
Exact energy                            : -0.9981493524136991
Energy difference (Stage 1 vs. exact)   : 4.19817403063405e-10
Energy difference (CUSP    vs. exact)   : 7.32798772662413e-07

Bond length                             : 2.0
CUSP optimized energy                   : -0.9486395378533693
Stage 1 energy                          : -0.94864097703

In [8]:
# Print the single-qubit circuit parameters
print(cusp_params)

{1.0: array([0.75489042, 0.11215268, 0.31254899]), 2.0: array([0.79097924, 0.36017615, 0.27696293]), 2.5: array([1.02528619, 0.43957161, 2.04261211]), 1.5: array([1.68265769, 1.76884209, 0.38514519])}


You have reached the end of our tutorial!  Thank you for participating!  If you have questions on the operation of this algorithm, please feel free to forward them to jonny@zapatacomputing.com.